In [1]:
## Libraries

import pandas as pd
import numpy as np
import json

In [2]:
with open('anonymized_collector_selector_data.json', 'r') as file:
    collector_data = json.load(file)

with open('anonymized_raw_conversations_data.json', 'r') as file:
    raw_data = json.load(file)
    
num = 0

candidates_id = [] # Save the candidates id into a list
questions = []

for key, value in collector_data.items():
    candidates_id.append(str(key))



In [3]:
candidates_id

['673dc174bbdffa3d88a4c57a804781e98f86d00b730fa829d832b9753e7a3779',
 '5cd3713008635b158efa7406b25eabc6825e93a0625df6e96dd31aa9d857bb8e',
 '17fbd58ca771f94008ffb056f7226dde097a47039fb1ec9bf32872f0b6d20625',
 'cee4404e5f694ad6141cb1dda30f78df1a5ed3e0efa6def532302a78a1ed26e8',
 '467b92deb304456a96d260db83d6fe421bcc30a5cf1113a7704de32a52301b10',
 '36f256f0c14392d372137b7874d33e347d0a948683093784dbbc3e49f02531de',
 'cd00dc29b5c5e5fb660945d12255f9a110e4d643a73059fbae2f9335ce8b8264',
 '60d1a00ead883aaca3c501c309e2cbf685f1cd61435241b2faeb1f3db82f7541',
 '8e8aa543ee96e42cb85626d06d12995c5f93cf0b92057a946d53598e694d1990',
 '417c5647e0cc51c44517b7bb1a44ba32a98a6bc78d17db7deb540531c776a4b8',
 '2ef48296f91c4a0fa9e55128121dd1141c8ab9ce9d7b1696acbe3d7acd821684',
 'b1eb364062dad98fbfa639a15f638f647b29fa286c5a9393244f4aa32ff3126a',
 '35e816d71c50d40dae1835d162ae6caf3e560c469809cc8b712d21e98906e457',
 'e31768282dac1712c54f44dcdc6411aa1b027986a9c5616b635ba3d1799bd176',
 'b4428b45723fa721ee1c54a3cf5d5b20

In [4]:
# Define an empty list or dictionaries to store the datasets
datasets = []
score =[]
question_answer_lists = {}
question_answer_listsencoded = {} # this one was created for keeping the altered answer and encoded answer separate
discarded = []
question_type = []
question_idL = []
conv_end = []

# Iterate through each client dictionary in the collection_data
first_dictionary = True  # Flag to check if it's the first dictionary
for client_id, client_data in collector_data.items():
  # Get the questions_answers dictionary
    score.append(client_data["score"])
    discarded.append(client_data["discarded"])   
    conv_end.append(client_data["who_end_conversation"])
    for question_id, question_data in client_data["question_answers"].items():
        if question_id not in question_answer_lists:
            question_answer_lists[question_id] = []
            question_answer_listsencoded[question_id] = []
        for key, value in question_data.items():
            if key == "answer":
                if value != "No especificado":
                    value = "Active response"
                question_answer_listsencoded[question_id].append(value)
                question_answer_lists[question_id].append(value)
        if first_dictionary == True and key == "type":
            question_type.append(value)
    first_dictionary = False  # Set flag to False after processing the first dictionary


# getting the questions ids
for key, data in question_answer_lists.items():
    question_idL.append(key)
                

# creating the dataframes
Candidatedf = pd.DataFrame()
Candidatedf["candidates_id"] = candidates_id



Candidatedf["score"] = score
Candidatedf["discarded"] = discarded
Candidatedf["conversation_end"] = conv_end

print(len(Candidatedf))

29


In [5]:
# number of questions responded by candidate
r_percan = []
for candidate, responses in raw_data.items():
    r_percan.append(len(responses))
r_percan

Candidatedf["total_conversation"] = r_percan

Candidatedf.head(6)
Candidatedf.to_csv('Candidatedf.csv', index=False) 


# Convert DataFrame to a list of dictionaries
Candidatedf = Candidatedf.to_dict(orient='records')

# Convert the list of dictionaries to JSON
Candidatedf = json.dumps(Candidatedf)

Candidatedf

file_path = 'Candidatedf.json'
with open(file_path, 'w') as json_file:
    json_file.write(Candidatedf)

In [6]:
question_df = pd.DataFrame()
question_df["question_id"] = question_idL
question_df["question_type"] = question_type

question_df.to_csv('question_df.csv', index=False) 


question_df = question_df.to_json(orient='records')

file_path = 'question_df.json'
with open(file_path, 'w') as json_file:
    json_file.write(question_df)
    
question_df

'[{"question_id":"6b121e51865742678437cf22fdef67bb","question_type":"Qualifying"},{"question_id":"8e39ba0b638a459aa5323aca5350f38c","question_type":"Qualifying"},{"question_id":"92636148a4a04227ac2a17c744ae4aa9","question_type":"Qualifying"},{"question_id":"92a6a5a99b694d398d1e591aaefe7558","question_type":"Disqualifier"},{"question_id":"9cc0d3e4f0054d6a930cfcc20ca2cd5a","question_type":"Informative"},{"question_id":"ff1d9a3711bc496c8a734171591a8fb7","question_type":"Disqualifier"}]'

In [7]:
c_id=[]
response=[]
q_id = []
for key, value in question_answer_listsencoded.items():
    for x in range(len(value)):
        c_id.append(candidates_id[x])
        response.append(value[x])
        q_id.append(key)
            
responses_df = pd.DataFrame()

responses_df["candidates_id"] = c_id
responses_df["question_id"] = q_id
responses_df["response"] = response

responses_df
responses_df.to_csv('responses_df.csv', index=False) 


responses_df = responses_df.to_json(orient='records')

file_path = 'responses_df.json'
with open(file_path, 'w') as json_file:
    json_file.write(responses_df)
    